# Fundamental Law of Memory Recall 



## Load packages

In [1]:
#!sudo apt-get install texlive-full  

In [2]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import scipy
from matplotlib import cm
import random
import heapq
from collections import Counter
import scipy as sp

#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')

## Model explanation

Item representations are chosen as random binary $\{0,1\}$ vectors where each element of the vector chosen to be 1 with small probability $f \ll 1$ independently of other elements.
Overlaps are defined as scalar products between these
representations.

The proposed recall process is based on two principles:

1. memory items are represented in the brain by overlapping random sparse neuronal ensembles in dedicated memory networks;
2. the next item to be recalled is the one with a largest overlap to the current one, excluding the item that was recalled on the previous step.

Pseudocode:
- find the second largest value in the first row of SM
- go to the row corresponding to its column index
- iterate 


## Define the functions

In [3]:
def BuildItems(L, N, f): 
    """
    L = number of items
    N = number of neurons
    f = probability of a neuron to be 1
    """
    np.random.seed(5) # good example with seed = 5, L = 5, N = 3000, f = 0.1
    return np.random.choice([0, 1], size=(N, L), p=[1-f, f])

def SparseRandomEnsemble(L, N, f):
    items = sp.sparse.random(N, L, density=f).A
    for i in range(N):
        for j in range(L):
            if items[i,j] != 0:
                items[i,j] = 1
    return items

def SimilarityMatrix(items):
    #sim = cosine_similarity(items, dense_output=True)
    sim = np.dot(items.T, items)
    # change the diagonal elements from 0 to 1 so that it does not interfere when we are
    # searching for the maximum element in a row
    np.fill_diagonal(sim, 0)
    return sim 

def PlotSM(sim, L):
    sns.set_context({"figure.figsize": (12, 12)})
    sns.set_style("white")
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    cmap = cm.get_cmap('Blues', 20)
    cmap.set_bad('w') # default value is 'k'
    ax1.imshow(sim, cmap=cmap)
    #plt.ylabel("Cluster", size = L)
    #plt.xlabel("Cluster", size = L)
    sns.despine()
    plt.colorbar(ax1.matshow(sim, cmap=cmap), shrink=.75)
    ax1.xaxis.set_label_position('bottom')
    ax1.xaxis.set_ticks_position('bottom')
    plt.xticks(range(0,L,1), size = L)
    plt.yticks(range(0,L,1), size = L)
    #plt.savefig('similarity_matrix.pdf', dpi=220, bbox_inches='tight')

def memory_recall_process(L, N, f, vb=False):
    items = BuildItems(L, N, f)
    sim = SimilarityMatrix(items)
    if vb == True: 
        print(sim)
    recall_list = []
    recall_list.append(0)
    recall_list.append(np.where(sim[0, ] == sim[0, ].max())[0][0])
    for i in range(1, L**2): 
        #if vb == True:
        #    print("Recall list =", recall_list)
        recall_list.append(np.where(sim[recall_list[i], ] == sim[recall_list[i], ].max())[0][0])
        if (sim[recall_list[i-1], recall_list[i]] == sim[recall_list[i], recall_list[i+1]]):
            sl = heapq.nlargest(2, sim[recall_list[i], ])
            if sl[0] == sl[1]:
                recall_list[i+1] = np.where(sim[recall_list[i], ] == sl[1])[0][1]
            else:
                recall_list[i+1] = np.where(sim[recall_list[i], ] == sl[1])[0][0]
        ############################################################################################
        # INSERT A CONDITION FOR STOPPING HERE:
        # if two previously visited items are retrieved in the same order then stop.
        ############################################################################################
    R = len(set(recall_list))
    return R

def TheoreticalScaling(L):
    return np.sqrt(1.5*np.pi*float(L))

## Run a small test

In [4]:
r = memory_recall_process(10, 3000, 0.1, vb = True)
r
# [0, 4, 9, 7, 2, 5, 3, 8, 6, 9] for random_seed=5

[[ 0 27 31 33 38 24 32 33 30 31]
 [27  0 29 27 28 31 27 26 26 26]
 [31 29  0 29 29 33 33 40 28 27]
 [33 27 29  0 25 36 29 32 34 30]
 [38 28 29 25  0 30 24 30 28 37]
 [24 31 33 36 30  0 28 27 18 25]
 [32 27 33 29 24 28  0 28 36 34]
 [33 26 40 32 30 27 28  0 31 37]
 [30 26 28 34 28 18 36 31  0 23]
 [31 26 27 30 37 25 34 37 23  0]]


9

## Do the actual computation

In [ ]:
N = 3000
f_val = [0.1, 0.05, 0.01]
L_val = [10, 20, 50, 80, 130, 280, 500]

nruns = 100

R_f01_L10 =  0
R_f01_L20 =  0
R_f01_L50 =  0 
R_f01_L80 =  0
R_f01_L130 = 0
R_f01_L280 = 0
R_f01_L500 = 0 

R_f005_L10 =  0
R_f005_L20 =  0
R_f005_L50 =  0
R_f005_L80 =  0
R_f005_L130 = 0
R_f005_L280 = 0
R_f005_L500 = 0 

R_f001_L10 =  0
R_f001_L20 =  0
R_f001_L50 =  0 
R_f001_L80 =  0
R_f001_L130 = 0
R_f001_L280 = 0
R_f001_L500 = 0 

for run in range(nruns):
    print("run ", run)
    print("f=0.1")
    R_f01_L10  = R_f01_L10  + memory_recall_process(L_val[0], N, f_val[0])
    R_f01_L20  = R_f01_L20  + memory_recall_process(L_val[1], N, f_val[0])
    R_f01_L50  = R_f01_L50  + memory_recall_process(L_val[2], N, f_val[0])
    R_f01_L80  = R_f01_L80  + memory_recall_process(L_val[3], N, f_val[0])
    R_f01_L130 = R_f01_L130 + memory_recall_process(L_val[4], N, f_val[0])
    R_f01_L280 = R_f01_L280 + memory_recall_process(L_val[5], N, f_val[0])
    R_f01_L500 = R_f01_L500 + memory_recall_process(L_val[6], N, f_val[0])
    print("f=0.05")
    R_f005_L10  = R_f005_L10  + memory_recall_process(L_val[0], N, f_val[1])
    R_f005_L20  = R_f005_L20  + memory_recall_process(L_val[1], N, f_val[1])
    R_f005_L50  = R_f005_L50  + memory_recall_process(L_val[2], N, f_val[1])
    R_f005_L80  = R_f005_L80  + memory_recall_process(L_val[3], N, f_val[1])
    R_f005_L130 = R_f005_L130 + memory_recall_process(L_val[4], N, f_val[1])
    R_f005_L280 = R_f005_L280 + memory_recall_process(L_val[5], N, f_val[1])
    R_f005_L500 = R_f005_L500 + memory_recall_process(L_val[6], N, f_val[1])
    print("f=0.01")
    R_f001_L10  = R_f001_L10  + memory_recall_process(L_val[0], N, f_val[2])
    R_f001_L20  = R_f001_L20  + memory_recall_process(L_val[1], N, f_val[2])
    R_f001_L50  = R_f001_L50  + memory_recall_process(L_val[2], N, f_val[2])
    R_f001_L80  = R_f001_L80  + memory_recall_process(L_val[3], N, f_val[2])
    R_f001_L130 = R_f001_L130 + memory_recall_process(L_val[4], N, f_val[2])
    R_f001_L280 = R_f001_L280 + memory_recall_process(L_val[5], N, f_val[2])
    R_f001_L500 = R_f001_L500 + memory_recall_process(L_val[6], N, f_val[2])

R_f01  = [R_f01_L10 , R_f01_L20 , R_f01_L50 , R_f01_L80 , R_f01_L130 , R_f01_L280 , R_f01_L500]
R_f005 = [R_f005_L10, R_f005_L20, R_f005_L50, R_f005_L80, R_f005_L130, R_f005_L280, R_f005_L500]
R_f001 = [R_f001_L10, R_f001_L20, R_f001_L50, R_f001_L80, R_f001_L130, R_f001_L280, R_f001_L500]

R_f01  = [elem / nruns for elem in R_f01]
R_f005 = [elem / nruns for elem in R_f005]
R_f001 = [elem / nruns for elem in R_f001]

## Plot

In [ ]:
plt.figure(figsize=(10,7))
plt.xlabel("List length")
plt.ylabel("Retrieved items (mean)")
plt.grid()
rlist = [R_f01, R_f005, R_f001]
plt.plot(L_val, [TheoreticalScaling(val) for val in L_val], '--', label='$\sqrt{3\pi L/2}$')
for r in range(3):
    plt.plot(L_val, rlist[r], 'o-', label='$f = '+str(f_val[r])+'$') 
plt.legend()
plt.savefig('symmetricSM.pdf', bbox_inches='tight')
plt.show()

## Backup & old code

The very first version of the `memory_recall_process` function.
```
def memory_recall_process_OLD(L, N, f):
    items = BuildItems(L, N, f)
    sim = SimilarityMatrix(items)
    # starting point: find the largest element in the first row
    recall_list = [None]*(L+1)
    recall_list[0] = np.where(sim[0, ] == sim[0, ].max())[0][0]
    for i in range(L):
        # find the largest element in a row
        recall_list[i+1] = np.where(sim[recall_list[i], ] == sim[recall_list[i], ].max())[0][0]
        # if the process points to an item that was just recalled in the previous step,
        # select the next largest overlap 
        if recall_list[i+1] == recall_list[i]:
            sl = heapq.nlargest(2, sim[recall_list[i], ])
            recall_list[i+1] = np.where(sim[recall_list[i], ] == sl[1])[0][0]
    R = len(set(recall_list))
    return R
```

Memory recall process starting from a random similarity matrix. Useless. 
```
def RandomSM_memory_recall_process(L):
    sim = np.random.rand(L, L)
    np.fill_diagonal(sim, 0)
    recall_list = [None]*(L+1)
    recall_list[0] = np.where(sim[0, ] == sim[0, ].max())[0][0]
    recall_list[1] = np.where(sim[recall_list[0], ] == sim[recall_list[0], ].max())[0][0]
    if sim[recall_list[1], recall_list[0]] == sim[recall_list[0], recall_list[1]]:
        sl = heapq.nlargest(2, sim[recall_list[0], ])
        recall_list[1] = np.where(sim[recall_list[0], ] == sl[1])[0][0]
    for i in range(1, L):
        # find the largest element in a row
        recall_list[i+1] = np.where(sim[recall_list[i], ] == sim[recall_list[i], ].max())[0][0]
        # if the process points to an item that was just recalled in the previous step,
        # select the next largest overlap 
        if sim[recall_list[i-1], recall_list[i]] == sim[recall_list[i], recall_list[i+1]]:
            sl = heapq.nlargest(2, sim[recall_list[i], ])
            recall_list[i+1] = np.where(sim[recall_list[i], ] == sl[1])[0][0]
    R = len(set(recall_list))
    return R
```

This is the old `memory_recall_process`, in which the `recall_list` array is allocated at the beginning, which for sure is not the best solution. 
```
def memory_recall_process(L, N, f):
    items = BuildItems(L, N, f)
    sim = SimilarityMatrix(items)
    print(sim)
    recall_list = [None]*(L**2)
    # always start from the first item
    recall_list[0] = 0
    # pick the most similar item to the first 
    recall_list[1] = np.where(sim[0, ] == sim[0, ].max())[0][0]
    #recall_list[2] = np.where(sim[recall_list[0], ] == sim[recall_list[0], ].max())[0][0]
    #if sim[recall_list[1], recall_list[0]] == sim[recall_list[0], recall_list[1]]:
    #    sl = heapq.nlargest(2, sim[recall_list[0], ])
    #    recall_list[1] = np.where(sim[recall_list[0], ] == sl[1])[0][0]
    for i in range(1, L):
        print("Recall list =", recall_list)
        # find the largest element in a row
        recall_list[i+1] = np.where(sim[recall_list[i], ] == sim[recall_list[i], ].max())[0][0]
        #print("i+1 before check", recall_list[i+1])
        # if the process points to an item that was just recalled in the previous step,
        # select the next largest overlap 
        #print("recall first:", recall_list[i+1])
        #print("if", recall_list[i+1], "==", recall_list[i])
        #print(sim[recall_list[i-1], recall_list[i]], "==", sim[recall_list[i], recall_list[i+1]])
        if (sim[recall_list[i-1], recall_list[i]] == sim[recall_list[i], recall_list[i+1]]):
            sl = heapq.nlargest(2, sim[recall_list[i], ])
            if sl[0] == sl[1]:
                #print("We have a problem...")
                #print("here:", np.where(sim[recall_list[i], ] == sl[1])[0][1])
                recall_list[i+1] = np.where(sim[recall_list[i], ] == sl[1])[0][1]
            else:
                recall_list[i+1] = np.where(sim[recall_list[i], ] == sl[1])[0][0]
            #print("after check:", recall_list[i+1])
    R = len(set(recall_list))
    return R
```

